In [13]:
# 라이브러리 import
! pip install bs4
! pip install pandas
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import pprint

In [14]:

### 중앙부처 데이터만 저장할 수 있도록 만든 코드입니다. 지자체나 지역별 데이터 저장을 원하면 수정하여 활용하시길 바랍니다.
## 1. 온통 청년 API로 정책 html 불러오기
key = 'f93ad94f49a4829e7739cc9f' ### 여기에 신청한 본인의 key를 넣어야 합니다!
page = ['0', '1', '2', '3', '4'] # 조회할 페이지, 총 정책 수가 업데이트로 인해 늘어나면 현재 '0' ~ '4'인 페이지를 추가해야 할 수도 있음. 예) page = ['0', '1', '2', '3', '4', '5']
outputNum = '100' # 출력건수, 기본 10 최대 100
#region = '003002015' # 지역별로 조회하고 싶다면 지역 코드를 지정하면 됨. 여러 지역을 동시에 조회하고 싶다면 쉼표(,)로 코드를 연결해 주면 됨. 예) '003002015,003002014'

contents = ''
for p in page:
    #queryParams = 'pageIndex=' + page + '&display=' + outputNum + '&openApiVlak=' + key + '&srchPolyBizSecd=' + region # 지역별로 조회하게 되면 page를 더 적게 지정해야 할 수도 있음.
    queryParams = 'pageIndex=' + p + '&display=' + outputNum + '&openApiVlak=' + key
    url = 'https://www.youthcenter.go.kr/opi/youthPlcyList.do?'+ queryParams

    # url 불러오기
    response = requests.get(url)
    #print(type(response))
    print(len(response.text)) # 130으로 나오는 경우 해당 페이지에 출력할 정책이 없는 경우임, 즉 130 이전 page까지만 포함시키면 됨.

    #데이터 값 출력해보기
    contents += response.text
    #print(type(contents)) # string 객체로 바뀜

245580
245580
240753
256758
120740


In [15]:
# 데이터 결과값 예쁘게 출력해주는 코드
pp = pprint.PrettyPrinter(indent=4, depth=None)
print(pp.pprint(contents[:3000]))

('<?xml version="1.0" encoding="UTF-8"?>\n'
 '<youthPolicyList>\n'
 '   <pageIndex>1</pageIndex>\n'
 '   <totalCnt>343</totalCnt>\n'
 '   <youthPolicy>\n'
 '      <rnum>1</rnum>\n'
 '      <bizId><![CDATA[R2024021319586]]></bizId>\n'
 '      <polyBizSecd>003002015011</polyBizSecd>\n'
 '      <polyBizTy>지자체</polyBizTy>\n'
 '      <polyBizSjnm><![CDATA[1인가구 동아리, 일(1)로 모여]]></polyBizSjnm>\n'
 '      <polyItcnCn><![CDATA[-]]></polyItcnCn>\n'
 '      <sporCn><![CDATA[지원내용- 양산시 가족센터 OT 및 활동 공유회 실시- 동아리 활동시 장소 제공 및 활동비 '
 '지원유의사항 1. 해당 프로그램은 우선 선발자 이후 선착순으로 접수되며, 정원 이후는 대기자로 선정될 수 있음.2. 당일 신청 취소 및 '
 '불참의 경우, 3개월 간 양산시가족센터프로그램 참여자 제한됨으로 유의. 문의사항 055-382-0988, 070-4354-4257 담당자 '
 '태민화]]></sporCn>\n'
 '      <sporScvl><![CDATA[-]]></sporScvl>\n'
 '      <bizPrdCn><![CDATA[-]]></bizPrdCn>\n'
 '      <prdRpttSecd><![CDATA[002004]]></prdRpttSecd>\n'
 '      <rqutPrdCn><![CDATA[2024.02.13~2024.02.23]]></rqutPrdCn>\n'
 '      <ageInfo><![CDATA[만 20세 ~ 60세]]></ageInfo>\n'
 '      <majrRqisCn><![CDAT

In [16]:
## 2. BeautifulSoup으로 html parsing
soup = BeautifulSoup(contents, 'html.parser')
#finded_values=soup.find_all('ageInfo')
#[x.text for x in finded_values] # .text를 통해 안의 내용물만을 불러옵니다

/Users/sukyungseok/jingi-chatbot/chatbot-using-langchain/.conda/lib/python3.11/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [17]:
print('현재 총 정책 수:', soup.find('totalcnt').text.replace('<totalcnt>', '').replace('</totalcnt>', ''))

현재 총 정책 수: 343


In [18]:
## 3. df 생성
YouthPolicyInfo = {}
# attr 이름 모두 소문자로
#attr_to_find_list=['polybizsjnm','sporcn', 'ageinfo','majrrqiscn','empmsttscn','splzrlmrqiscn','accrrqiscn','prcpcn','aditrscn','prcplmtttrgtcn']
attr_to_find_list=['rnum', 'bizid', 'polybizsecd', 'polybizty', 'polybizsjnm', 'polyitcncn', 'sporcn', 'sporscvl', 'bizprdcn', 'prdrpttsecd', 'rqutprdcn', 
                   'ageinfo','majrrqiscn','empmsttscn','splzrlmrqiscn','accrrqiscn','prcpcn','aditrscn','prcplmtttrgtcn',
                   'rqutproccn', 'pstnpaprcn', 'jdgnprescn', 'rquturla', 'rfcsiteurla1', 'rfcsiteurla2', 'mngtmson', 'mngtmrofchercn', 'cherctpccn',
                   'cnsgnmor', 'tintchercn', 'tintcherctpccn', 'etct', 'polyrlmcd'] # 모두 소문자여야 함.
print(len(attr_to_find_list))

for each_attr in attr_to_find_list:
    finded_attr=soup.find_all(each_attr)
    YouthPolicyInfo[each_attr]=[x.text for x in finded_attr]

df = pd.DataFrame(YouthPolicyInfo)
#alt_colnames = ['정책명', '지원내용', '연령', '전공요건', '취업상태', '특화분화', '학력', '거주지및소득', '추가단서', '참여제한']
alt_colnames = ['번호', '정책 ID', '정책 일련번호', '기관 및 지자체 구분', '정책명', '정책 소개', '지원 내용', '지원 규모', '사업 운영 기간', '사업 신청 기간 반복 구분 코드', '사업 신청 기간', 
                '연령', '전공', '취업 상태', '특화 분화', '학력', '거주지 및 소득', '추가 단서 사항', '참여 제한 대상', 
                '신청 절차', '제출 서류', '심사 발표', '신청 사이트 주소', '참고 사이트 주소1', '참고 사이트 주소2', '주관 부처명', '주관 부처 담당자 이름', '주관 부처 담당자 연락처',
                '운영 기관명', '운영 기관 담당자 이름', '운영 기관 담당자 연락처', '기타 사항', '정책 분야 코드']
print(len(alt_colnames))

df.columns = alt_colnames

33
33


In [19]:
print(df.shape)
df.head()

(443, 33)


,번호,정책 ID,정책 일련번호,기관 및 지자체 구분,정책명,정책 소개,지원 내용,지원 규모,사업 운영 기간,사업 신청 기간 반복 구분 코드,...,참고 사이트 주소1,참고 사이트 주소2,주관 부처명,주관 부처 담당자 이름,주관 부처 담당자 연락처,운영 기관명,운영 기관 담당자 이름,운영 기관 담당자 연락처,기타 사항,정책 분야 코드
0,1,R2024021319586,003002015011,지자체,"1인가구 동아리, 일(1)로 모여",-,지원내용- 양산시 가족센터 OT 및 활동 공유회 실시- 동아리 활동시 장소 제공 및...,-,-,002004,...,-,-,양산시가족센터,-,055-382-0988,주관기관과 동일,-,-,-,023040
1,2,R2024021319566,003002015,지자체,2024 경상남도 청년봉사단 신규 단원 모집,-,모집기간 : 1월 31일(수)~2월 24일(토) 모집대상 : 도내 거주 또는 재직중...,-,-,002005,...,-,-,경상남도자원봉사센터,-,055-281-1365,주관기관과 동일,-,-,-,023040
2,3,R2024021319565,003001001,중앙부처,청년일자리도약장려금 사업(2024),5인 이상 우선지원대상기업에서 청년을 정규직으로 채용하고 6개월 이상 고용 유지할 ...,"- 신규 채용 청년 1인당 월 최대 60만원씩 1년간 지원, 최초 채용 후 2년 근...",- (지원 한도) 사업참여 신청 직전 월말부터 이전 1년간 평균 고용보험 피보험자 ...,null,002001,...,null,null,고용노동부,null,null,지역별 운영 기관 별도 확인,null,null,null,023010
3,4,R2024021319564,003001007,중앙부처,청년월세한시특별지원,월 최대 20만원씩 12개월 분의 월세를 지원받을 수 있는 청년월세 특별지원(예산 ...,"□ 월 최대 20만원, 최대 12개월 분할 지급(평생 1회 지급)\r\n□ 무주택 ...",null,null,002001,...,null,null,국토교통부,null,null,-,null,null,null,023020
4,5,R2024021319583,003002014,지자체,2024 경북 청년CEO 심화육성 지원사업,유망 창업아이템을 보유한 경북도 내 창업기업을 대상으로 체계적인 지원 및 창업성공률...,"□ 프로그램 : 창업실무교육, 전문가 컨설팅 및 네트워킹 등\r\n- (창업실무교육...",□ 청년CEO(심화과정) 20명\r\n□ 청년CEO(일반과정) 10명,2024-01-01~2024-12-31,002004,...,http://gbyouth.co.kr/policy/list.tc?mn=2379&pa...,-,(재)경북테크노파크,-,연락처 : 053) 819-3073~72\r\nE-Mail : shlee@gbtp....,(재)경북테크노파크 경북청년창업지원센터,-,연락처 : 053) 819-3073~72\r\nE-Mail : shlee@gbtp....,-,023010


In [20]:
## 4. 부족한 정보 추가
# 4-1. 정책 분야 추가
polysec = []
for code in df['정책 분야 코드']:
    if code == '023010':
        polysec.append('일자리 분야')
    elif code == '023020':
        polysec.append('주거 분야')
    elif code == '023030':
        polysec.append('교육 분야')
    elif code == '023040':
        polysec.append('복지·문화 분야')
    elif code == '023050':
        polysec.append('참여·권리 분야')
    else:
        polysec.append('코드 매칭 분야 없음')

df['정책 분야'] = polysec

In [21]:
# 4-2. 온통 청년 url 추가
df['온통 청년 url'] = "https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?pageIndex=1&frameYn=&bizId=" + df['정책 ID'] + "&dtlOpenYn=&plcyTpOpenTy=&plcyCmprInfo=&srchWord=&chargerOrgCdAll=&srchAge=&trgtJynEmp=&trgtJynEmp=&srchSortOrder=1&pageUnit=12"

In [22]:
print(df.shape)
df.head()

(443, 35)


,번호,정책 ID,정책 일련번호,기관 및 지자체 구분,정책명,정책 소개,지원 내용,지원 규모,사업 운영 기간,사업 신청 기간 반복 구분 코드,...,주관 부처명,주관 부처 담당자 이름,주관 부처 담당자 연락처,운영 기관명,운영 기관 담당자 이름,운영 기관 담당자 연락처,기타 사항,정책 분야 코드,정책 분야,온통 청년 url
0,1,R2024021319586,003002015011,지자체,"1인가구 동아리, 일(1)로 모여",-,지원내용- 양산시 가족센터 OT 및 활동 공유회 실시- 동아리 활동시 장소 제공 및...,-,-,002004,...,양산시가족센터,-,055-382-0988,주관기관과 동일,-,-,-,023040,복지·문화 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
1,2,R2024021319566,003002015,지자체,2024 경상남도 청년봉사단 신규 단원 모집,-,모집기간 : 1월 31일(수)~2월 24일(토) 모집대상 : 도내 거주 또는 재직중...,-,-,002005,...,경상남도자원봉사센터,-,055-281-1365,주관기관과 동일,-,-,-,023040,복지·문화 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
2,3,R2024021319565,003001001,중앙부처,청년일자리도약장려금 사업(2024),5인 이상 우선지원대상기업에서 청년을 정규직으로 채용하고 6개월 이상 고용 유지할 ...,"- 신규 채용 청년 1인당 월 최대 60만원씩 1년간 지원, 최초 채용 후 2년 근...",- (지원 한도) 사업참여 신청 직전 월말부터 이전 1년간 평균 고용보험 피보험자 ...,null,002001,...,고용노동부,null,null,지역별 운영 기관 별도 확인,null,null,null,023010,일자리 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
3,4,R2024021319564,003001007,중앙부처,청년월세한시특별지원,월 최대 20만원씩 12개월 분의 월세를 지원받을 수 있는 청년월세 특별지원(예산 ...,"□ 월 최대 20만원, 최대 12개월 분할 지급(평생 1회 지급)\r\n□ 무주택 ...",null,null,002001,...,국토교통부,null,null,-,null,null,null,023020,주거 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
4,5,R2024021319583,003002014,지자체,2024 경북 청년CEO 심화육성 지원사업,유망 창업아이템을 보유한 경북도 내 창업기업을 대상으로 체계적인 지원 및 창업성공률...,"□ 프로그램 : 창업실무교육, 전문가 컨설팅 및 네트워킹 등\r\n- (창업실무교육...",□ 청년CEO(심화과정) 20명\r\n□ 청년CEO(일반과정) 10명,2024-01-01~2024-12-31,002004,...,(재)경북테크노파크,-,연락처 : 053) 819-3073~72\r\nE-Mail : shlee@gbtp....,(재)경북테크노파크 경북청년창업지원센터,-,연락처 : 053) 819-3073~72\r\nE-Mail : shlee@gbtp....,-,023010,일자리 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...


In [23]:
# 정책 분야 카테고리 확인
df['정책 분야'].unique()

array(['복지·문화 분야', '일자리 분야', '주거 분야', '참여·권리 분야', '교육 분야'], dtype=object)

In [26]:
## 5. 중복 데이터 제거
# 5-1. 중복 데이터 확인
print(df['정책명'].value_counts())
#df['정책명'].unique()

정책명
충북행복결혼공제                   3
1인가구 동아리, 일(1)로 모여         2
입영지원금 지원(광주시)              2
청년 올라프로그램(광주시)             2
유스놀이터 운영(광주시)              2
                          ..
인천논현4 1블록 창업지원주택 입주자 모집    1
청년 정신건강 조기중재사업             1
인천 마이스(MICE)업체 청년인턴십 운영    1
인천시 동구일자리센터 카카오톡 채널 운영     1
구직청년 면접 정장 대여              1
Name: count, Length: 341, dtype: int64


In [27]:
# 5-2. 중복 제거
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(443, 35)
(343, 35)


In [28]:
## 6. 중앙부처 데이터만 가져오기
# 6-1. 중앙부처/지자체 구분 확인
df['기관 및 지자체 구분'].unique()

array(['지자체', '중앙부처'], dtype=object)

In [29]:
# 6-2. 중앙부처 정책만 있는 df 생성
df_head = df[df['기관 및 지자체 구분']=='중앙부처']

In [30]:
# 7. csv 저장하기 
df_head.to_csv("/Users/sukyungseok/jingi-chatbot/chatbot-using-langchain/data/중앙부처.csv")